# Задание 1. Загрузка данных
Изучить представленный набор данных на основе описания его столбцов, загрузить его и оставить 8 столбцов для дальнейшего изучения: surgery?, Age, rectal temperature, pulse, respiratory rate, temperature of extremities, pain, outcome.

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('horse_data.csv', header=None)
data.head()
data.columns = range(1,29)
data = data[[1,2,4,5,6,7,11,23]]
data.columns=['surgery','age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']

In [2]:
data.head(10)

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.50,66,28,3,5,2
1,1,1,39.2,88,20,?,3,3
2,2,1,38.30,40,24,1,3,1
3,1,9,39.10,164,84,4,2,2
4,2,1,37.30,104,35,?,?,2
5,2,1,?,?,?,2,2,1
6,1,1,37.90,48,16,1,3,1
7,1,1,?,60,?,3,?,2
8,2,1,?,80,36,3,4,3
9,2,9,38.30,90,?,1,5,1


# Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [3]:
data['surgery'] = pd.to_numeric(data['surgery'], errors='coerce')
data['age'] = pd.to_numeric(data['age'], errors='coerce')
data['rectal temperature'] = pd.to_numeric(data['rectal temperature'], errors='coerce')
data['pulse'] = pd.to_numeric(data['pulse'], errors='coerce')
data['respiratory rate'] = pd.to_numeric(data['respiratory rate'], errors='coerce')
data['temperature of extremities'] = pd.to_numeric(data['temperature of extremities'], errors='coerce')
data['pain'] = pd.to_numeric(data['pain'], errors='coerce')
data['outcome'] = pd.to_numeric(data['outcome'], errors='coerce')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     299 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


## Операции

In [5]:
data['surgery'].value_counts() # была операция - 180, не было операции - 118

1.0    180
2.0    119
Name: surgery, dtype: int64

In [6]:
print('Мода:',data['surgery'].mode()[0])
print('Медиана:', data['surgery'].quantile(0.5))

Мода: 1.0
Медиана: 1.0


Вывод: 60% лошадей оперируются после поступления. 

## Возраст

In [7]:
data['age'].value_counts()

1    276
9     24
Name: age, dtype: int64

In [8]:
print('Мода:',data['age'].mode()[0])
print('Медиана:', data['age'].quantile(0.5))

Мода: 1
Медиана: 1.0


In [9]:
print(data.loc[data.age == 9,'surgery'].value_counts(normalize=True))
print (data.loc[data.age == 1,'surgery'].value_counts(normalize=True))

1.0    0.75
2.0    0.25
Name: surgery, dtype: float64
1.0    0.589091
2.0    0.410909
Name: surgery, dtype: float64


Вывод: По нашим данным в столбце могут быть значения только  1 = Adult horse или 2 = Young (< 6 months), поэтому считаем, что 9 это опечатка и нужно заменить значение на 2. Кажется, если лошадь меньше 6 месяцев, то вероятность операции возрастает, но выборка слишком маленькая, чтобы можно было сделать точные выводы.

## Температура

In [10]:
print(data['rectal temperature'].agg(['min','max']))

min    35.4
max    40.8
Name: rectal temperature, dtype: float64


In [11]:
print('Мода:',data['rectal temperature'].mode()[0])
print('Медиана:', data['rectal temperature'].quantile(0.5))


iqr = data['rectal temperature'].quantile(0.75) - data['rectal temperature'].quantile(0.25)
print('Отклонение:',data['rectal temperature'].std() )
print('Нижний выброс:',data['rectal temperature'].quantile(0.25) - (1.5 * iqr))
print('Верхний выброс:', data['rectal temperature'].quantile(0.75) + (1.5 * iqr))

Мода: 38.0
Медиана: 38.2
Отклонение: 0.7322886641121578
Нижний выброс: 36.74999999999999
Верхний выброс: 39.550000000000004


Вывод: Нормальная температура 37,8, чаще всего на прием приходят с лошадьми у которых чуть повышенная температура - 38.

## Пульс

In [12]:
print(data['pulse'].agg(['min','max']))

min     30.0
max    184.0
Name: pulse, dtype: float64


In [13]:
print('Мода:',data['pulse'].mode()[0])
print('Медиана:', data['pulse'].quantile(0.5))


iqr = data['pulse'].quantile(0.75) - data['pulse'].quantile(0.25)
print('Отклонение:',data['pulse'].std() )
print('Нижний выброс:',data['pulse'].quantile(0.25) - (1.5 * iqr))
print('Верхний выброс:', data['pulse'].quantile(0.75) + (1.5 * iqr))

Мода: 48.0
Медиана: 64.0
Отклонение: 28.630556660735003
Нижний выброс: -12.0
Верхний выброс: 148.0


Вывод: Нормальный пульс считается 30-40 для взрослых лошадей, у маленьких может быть понижен. В целом повышенный пульс свидетельствуют о болезненности. В нашей выборке лошади чаще всего с повышенным пульсом.

##  Частота дыхания

In [14]:
print(data['respiratory rate'].agg(['min','max']))

min     8.0
max    96.0
Name: respiratory rate, dtype: float64


In [15]:
print('Мода:',data['respiratory rate'].mode()[0])
print('Медиана:', data['respiratory rate'].quantile(0.5))


iqr = data['pulse'].quantile(0.75) - data['respiratory rate'].quantile(0.25)
print('Отклонение:',data['respiratory rate'].std() )
print('Нижний выброс:',data['respiratory rate'].quantile(0.25) - (1.5 * iqr))
print('Верхний выброс:', data['respiratory rate'].quantile(0.75) + (1.5 * iqr))

Мода: 20.0
Медиана: 24.5
Отклонение: 17.642231385134664
Нижний выброс: -85.75
Верхний выброс: 140.25


## Температура конечностей

In [16]:
data['temperature of extremities'].value_counts()

3.0    109
1.0     78
2.0     30
4.0     27
Name: temperature of extremities, dtype: int64

In [17]:
print('Мода:',data['temperature of extremities'].mode()[0])
print('Медиана:', data['temperature of extremities'].quantile(0.5))

Мода: 3.0
Медиана: 3.0


Вывод: Чаще всего встречается повышенная температура. Она должна коррелировать с температурой тела.

## Боль

In [18]:
data['pain'].value_counts()

3.0    67
2.0    59
5.0    42
4.0    39
1.0    38
Name: pain, dtype: int64

In [19]:
print('Мода:',data['pain'].mode()[0])
print('Медиана:', data['pain'].quantile(0.5))

Мода: 3.0
Медиана: 3.0


Вывод: Чаще всего встречается периодическая легкая боль. Кажется уровень боли должен коррелировать с операциями.

##  Смерть

In [20]:
data['outcome'].value_counts()

1.0    178
2.0     77
3.0     44
Name: outcome, dtype: int64

In [21]:
data.loc[data.surgery == 1,'outcome'].value_counts(normalize=True)

1.0    0.527778
2.0    0.322222
3.0    0.150000
Name: outcome, dtype: float64

Вывод: 52% после операций поправляются. 

# Задание 3. Работа с пропусками
Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать.

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     299 non-null    float64
 1   age                         300 non-null    int64  
 2   rectal temperature          240 non-null    float64
 3   pulse                       276 non-null    float64
 4   respiratory rate            242 non-null    float64
 5   temperature of extremities  244 non-null    float64
 6   pain                        245 non-null    float64
 7   outcome                     299 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 18.9 KB


In [24]:
data.isna().sum()

surgery                        1
age                            0
rectal temperature            60
pulse                         24
respiratory rate              58
temperature of extremities    56
pain                          55
outcome                        1
dtype: int64

Вывод: <br>
0 surgery - имеет 1 пропуск - нужно посмотреть какие данные отсутствуют и принять решение:  удалить или заполнить медианой по группе<br>
1 age - нет пропусков<br>
2 rectal temperature - 60 пропусков. Нужно посмотреть какие данные отсутствуют и принять решение: удалить или заполнить медианой по группе<br>
3 pulse - 24 пропуска. Нужно посмотреть какие данные отсутствуют и принять решение: удалить или заполнить медианой по группе<br>
4 respiratory rate - 58 пропусков, так как данный столбец сомнительно полезный из-за большой частоты колебаний, можно заполнить медианой<br>
5 temperature of extremities - 56 пропусков. Можно заполнить медианой по группе, так как значение коррелирует с температурой тела<br>
6 pain - субъективные данные об уровне боли лошади, можно заполить медианой<br>
7 outcome  - 1 пропуск, думаю можно удалить, так как показатель выжный и может исказить результаты анализы при заполнении похожими/средними.<br>

Удалим данные, где отсутсвуют 5 значений из 7

In [25]:
data.dropna(thresh=5, inplace=True)

## Операции

In [26]:
data[data['surgery'].isna() == True]

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
132,NaN,1,38.0,48.0,20.0,3.0,4.0,NaN


Так как отсутсвует важная информация по наличию операции и смертности, то имеет смысл удалить строчку, чтобы не искажать данные

In [27]:
data.dropna(subset = ['surgery'], inplace = True)

## Возраст

In [28]:
data.replace(9, 2, inplace = True)

Заменяем значение 9 на 2.

## Температура

Заполним данные по группам: 
1) У кого не было операции и кто выжил заполним медианой такой же группы


In [29]:
data.loc[(data['surgery']== 2) & (data['outcome']== 1) ,'rectal temperature'].median()

38.25

In [30]:
data.loc[(data['surgery']== 2) & (data['outcome']== 1) & (data['rectal temperature'].isna() == True),'rectal temperature'] = 38.25

2) Заполним данные по группа исходя из температуры конечностей

In [31]:
print(data.groupby(['temperature of extremities'])['rectal temperature'].median())

temperature of extremities
1.0    38.200
2.0    38.225
3.0    38.150
4.0    38.350
Name: rectal temperature, dtype: float64


In [32]:
data.loc[(data['temperature of extremities'] == 1) & (data['rectal temperature'].isna() == True),'rectal temperature'] = 38.2
data.loc[(data['temperature of extremities'] == 2) & (data['rectal temperature'].isna() == True),'rectal temperature'] = 38.2
data.loc[(data['temperature of extremities'] == 3) & (data['rectal temperature'].isna() == True),'rectal temperature'] = 38.1
data.loc[(data['temperature of extremities'] == 4) & (data['rectal temperature'].isna() == True),'rectal temperature']= 38.35

3) Оставшиеся данные удалим, так как нет значений между которыми существует корреляция

In [33]:
data[data['rectal temperature'].isna() == True]

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
87,2.0,1,NaN,40.0,16.0,NaN,NaN,3.0
169,1.0,1,NaN,65.0,24.0,NaN,5.0,3.0
177,1.0,1,NaN,68.0,14.0,NaN,4.0,2.0


In [34]:
data.dropna(subset = ['rectal temperature'], inplace = True)

## Пульс

Так как пульс коррелирует с общим состоянием животного, то будем заполнять медианой по группам от температуры конечностей (в идеале использовать температуру тела и разбить на много групп, чтобы для каждой заполнить свой пульс, но в рамках текущей задачи и если учитывать, что температура конечностей коррелирует с температурой тела, этого будет достаточно)

In [35]:
print(data.groupby('temperature of extremities')['pulse'].quantile(0.25))
print(data.groupby('temperature of extremities')['pulse'].quantile(0.5))
print(data.groupby('temperature of extremities')['pulse'].quantile(0.75))

temperature of extremities
1.0    48.0
2.0    48.0
3.0    60.0
4.0    72.0
Name: pulse, dtype: float64
temperature of extremities
1.0    54.0
2.0    56.0
3.0    80.0
4.0    90.0
Name: pulse, dtype: float64
temperature of extremities
1.0     66.0
2.0     68.0
3.0     97.5
4.0    120.0
Name: pulse, dtype: float64


In [37]:
data.loc[(data['temperature of extremities'] ==1 ) & (data['pulse'].isna() == True),'pulse'] = 53.0
data.loc[(data['temperature of extremities'] ==2 ) & (data['pulse'].isna() == True),'pulse'] = 56.0
data.loc[(data['temperature of extremities'] ==3 ) & (data['pulse'].isna() == True),'pulse'] = 80.0
data.loc[(data['temperature of extremities'] ==4 ) & (data['pulse'].isna() == True),'pulse'] = 88.0

In [38]:
data[data['pulse'].isna() == True]

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome


## Частота дыхания

In [39]:
data['respiratory rate'].fillna(data['respiratory rate'].median(), inplace=True)

## Температура конечностей

In [40]:
print(data.groupby('temperature of extremities')['rectal temperature'].quantile(0.25))
print(data.groupby('temperature of extremities')['rectal temperature'].quantile(0.5))
print(data.groupby('temperature of extremities')['rectal temperature'].quantile(0.75))

temperature of extremities
1.0    37.800
2.0    37.925
3.0    37.975
4.0    38.200
Name: rectal temperature, dtype: float64
temperature of extremities
1.0    38.20
2.0    38.20
3.0    38.10
4.0    38.35
Name: rectal temperature, dtype: float64
temperature of extremities
1.0    38.4
2.0    38.6
3.0    38.5
4.0    38.6
Name: rectal temperature, dtype: float64


Заполним температуру конечностей исходя из промежутков температуры тела

In [43]:
data.loc[(data['rectal temperature'] >= 36.00) & (data['rectal temperature'] <= 38.10)& (data['temperature of extremities'].isna() == True),'temperature of extremities'] = 3
data.loc[(data['rectal temperature'] > 38.10) & (data['rectal temperature'] <= 38.15)& (data['temperature of extremities'].isna() == True),'temperature of extremities'] = 1
data.loc[(data['rectal temperature'] > 38.15) & (data['rectal temperature'] <= 38.20)& (data['temperature of extremities'].isna() == True),'temperature of extremities'] = 2
data.loc[(data['rectal temperature'] > 38.20) & (data['rectal temperature'] <= 40.00)& (data['temperature of extremities'].isna() == True),'temperature of extremities'] = 4

In [44]:
data[data['temperature of extremities'].isna() == True]

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome


## Боль

In [45]:
print(data.groupby('surgery')['pain'].median())

surgery
1.0    3.0
2.0    2.0
Name: pain, dtype: float64


In [46]:
#Запросы с loc и fillna и группировой и fillna не работают, не подскажите почему?
#data.loc[data['surgery']==1,'pain'].fillna(data.data['surgery']==1].median(), inplace=True)
#data['pain'].fillna(data.groupby('surgery')['pain'].median(), inplace=True)

In [47]:
data.loc[(data['surgery']==1) & (data['pain'].isna() == True),'pain'] = 3
data.loc[(data['surgery']==2) & (data['pain'].isna() == True),'pain'] = 2

##  Смерть

In [48]:
data[data['outcome'].isna() == True]

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome


Данный пропуск был удален ранее

## Финальный датафрейм

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 286 entries, 0 to 299
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   surgery                     286 non-null    float64
 1   age                         286 non-null    int64  
 2   rectal temperature          286 non-null    float64
 3   pulse                       286 non-null    float64
 4   respiratory rate            286 non-null    float64
 5   temperature of extremities  286 non-null    float64
 6   pain                        286 non-null    float64
 7   outcome                     286 non-null    float64
dtypes: float64(7), int64(1)
memory usage: 20.1 KB


In [50]:
data

,surgery,age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2.0,1,38.50,66.0,28.0,3.0,5.0,2.0
1,1.0,1,39.20,88.0,20.0,4.0,3.0,3.0
2,2.0,1,38.30,40.0,24.0,1.0,3.0,1.0
3,1.0,2,39.10,164.0,84.0,4.0,2.0,2.0
4,2.0,1,37.30,104.0,35.0,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...
295,1.0,1,38.35,120.0,70.0,4.0,2.0,3.0
296,2.0,1,37.20,72.0,24.0,3.0,4.0,3.0
297,1.0,1,37.50,72.0,30.0,4.0,4.0,2.0
298,1.0,1,36.50,100.0,24.0,3.0,3.0,1.0
